In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
import cv2

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# loading dataset

def load_data(path , split = 0.1):
    
    images = sorted(glob(os.path.join(path , "PNG/Original/*")))
    
    masks = sorted(glob(os.path.join(path , "PNG/Ground Truth/*")))
    
    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    
    # 612 total 
    # 61 test
    # 61 validate
    
    train_x , valid_x = train_test_split(images , test_size = valid_size, random_state = 42)
    
    train_y , valid_y = train_test_split(masks , test_size = valid_size, random_state = 42)
    
    
    train_x , test_x = train_test_split(train_x , test_size = test_size, random_state = 42)
    
    train_y , test_y = train_test_split(train_y , test_size = test_size, random_state = 42)
    
    return (train_x , train_y ) , (valid_x , valid_y) , (test_x , test_y)

In [4]:
def read_image(path):
    
    path = path.decode()
    x = cv2.imread(path , cv2.IMREAD_COLOR)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256 , 3
    return x


In [5]:
def read_mask(path):
    
    path = path.decode()
    x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256
    x = np.expand_dims(x, axis = -1)
    # size is 256, 256 , 1
    
    return x

In [6]:
def tf_parse (x,y) :
    
    def _parse(x,y):
        
        x = read_image(x)
        y = read_mask(y)
        
        return x,y
    x,y = tf.numpy_function(_parse , [x,y] , [tf.float64 , tf.float64])
    
    x.set_shape([256,256,3])
    y.set_shape([256,256,1])
    
    return x,y

In [7]:
def tf_dataset(x,y,batch = 8):
    
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [8]:
if __name__ == "__main__":
    
    path = "/kaggle/input/cvcclinicdb"
    (train_x , train_y ) , (valid_x , valid_y) , (test_x , test_y) = load_data(path,0.1)
    
    # 612 total 
    # 61 test
    # 61 validate
    # 490 train
    
    ds = tf_dataset(test_x , test_y)
    
    for x, y in ds:
        
        print(x.shape , y.shape) # batch of 8 images with 8 img and 8 masks
        break

(8, 256, 256, 3) (8, 256, 256, 1)


In [9]:

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model

def conv_block(x, num_filters):
    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
 
    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
 
    return x

def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    p = L.MaxPool2D((2, 2))(x)
    return x, p

def attention_gate(g, s, num_filters):
    Wg = L.Conv2D(num_filters, 1, padding="same")(g)
    Wg = L.BatchNormalization()(Wg)
 
    Ws = L.Conv2D(num_filters, 1, padding="same")(s)
    Ws = L.BatchNormalization()(Ws)
 
    out = L.Activation("relu")(Wg + Ws)
    out = L.Conv2D(num_filters, 1, padding="same")(out)
    out = L.Activation("sigmoid")(out)
 
    return out * s


def decoder_block(x, s, num_filters):
    x = L.UpSampling2D(interpolation="bilinear")(x)
    s = attention_gate(x, s, num_filters)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters)
    return x


def attention_unet(input_shape):
    """ Inputs """
    inputs = L.Input(input_shape)
 
    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
 
    b1 = conv_block(p3, 512)
 
    """ Decoder """
    d1 = decoder_block(b1, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)
 
    """ Outputs """
    outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d3)
 
    """ Model """
    model = Model(inputs, outputs, name="Attention-UNET")
    return model

if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = attention_unet(input_shape)
    model.summary()


Model: "Attention-UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                    

In [10]:

import tensorflow.keras.backend as K

from sklearn.metrics import jaccard_score,confusion_matrix

def IoU_coef(y_true, y_pred):
    
    print(y_true.shape)
    print(y_pred.shape)
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def dice_coef(y_true, y_pred):
    
    print(y_true.shape)
    print(y_pred.shape)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

In [11]:
from tensorflow import keras
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf


In [12]:
# Cross Entropy
# Focal 
# Dice 
# Tversky
# Tversky Focal
# Combo
# Unified Focal (Sym)
# Unified Focal (Asym)


In [13]:
# dice_loss = DiceLoss()
# binary_focal_loss = BinaryFocalLoss()
# combo_loss = binary_crossentropy + dice_loss


In [14]:

def dice_coef(y_true, y_pred, smooth=100):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [15]:
def sym_unified_focal_loss(y_true, y_pred ):
    
    axis = identify_axis(y_true.get_shape())
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    def loss_function(y_true,y_pred):
        weight=0.5
        delta=0.6 
        gamma=0.5
        symmetric_ftl = symmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        symmetric_fl = symmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        if weight is not None:
            return (weight * symmetric_ftl) + ((1-weight) * symmetric_fl)
        else:
            return symmetric_ftl + symmetric_fl

    return loss_function(y_true,y_pred)

In [16]:
def iou(y_true, y_pred) :
    
    def f (y_true, y_pred) :
        
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    
    return tf.numpy_function(f, [y_true , y_pred] , tf.float32)
        
      

In [17]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint , ReduceLROnPlateau , CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision
from keras import optimizers
from tensorflow.keras.losses import BinaryFocalCrossentropy
  
        
if __name__ == "__main__":
    
    
    np.random.seed(42)
    tf.random.set_seed(42)
    batch = 16
    lr = 1e-2
    epochs = 100
    
    train_dataset= tf_dataset(train_x , train_y, batch = batch)
    valid_dataset= tf_dataset(valid_x , valid_y, batch = batch)
    
    
    model = attention_unet((256, 256, 3))
    
    binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()
    opt = tf.keras.optimizers.Adam(lr)
    
    metrics = ["acc" , Recall() , Precision() , iou , dice_coef]
    
    
    model.compile(loss = binary_cross_entropy , optimizer = opt , metrics = metrics)
    
    callbacks  = [
        ModelCheckpoint("/kaggle/working/model.h5"),
        ReduceLROnPlateau(monitor = "val_loss" , factor = 0.5 , patience = 5),
        EarlyStopping(monitor = "val_loss" , patience = 30 , restore_best_weights = False)
    ]
    
    
    train_steps = len(train_x) // batch
    valid_steps = len(valid_x) // batch
    
    if len(train_x) % batch != 0 :
        train_steps+= 1
        
    if len(valid_x) % batch != 0 :
        valid_steps+= 1


    model.fit(
        
        train_dataset, 
        validation_data = valid_dataset,
        epochs = epochs,
        steps_per_epoch = train_steps,
        validation_steps = valid_steps,
        callbacks = callbacks,
        shuffle = False
    )
    
    

Epoch 1/100
31/31 [==============================] - 61s 908ms/step - loss: 0.3200 - acc: 0.8758 - recall: 0.0695 - precision: 0.1603 - iou: 0.0947 - dice_coef: 0.1726 - val_loss: 5891.2104 - val_acc: 0.9039 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou: 1.1023e-20 - val_dice_coef: 0.0011 - lr: 0.0100
Epoch 2/100
31/31 [==============================] - 16s 525ms/step - loss: 0.2351 - acc: 0.9038 - recall: 0.0677 - precision: 0.4876 - iou: 0.1369 - dice_coef: 0.2397 - val_loss: 245.2926 - val_acc: 0.1189 - val_recall: 1.0000 - val_precision: 0.0909 - val_iou: 0.0887 - val_dice_coef: 0.1627 - lr: 0.0100
Epoch 3/100
31/31 [==============================] - 17s 542ms/step - loss: 0.2284 - acc: 0.9061 - recall: 0.1444 - precision: 0.5407 - iou: 0.1546 - dice_coef: 0.2667 - val_loss: 5.4626 - val_acc: 0.5104 - val_recall: 0.9739 - val_precision: 0.1502 - val_iou: 0.1405 - val_dice_coef: 0.2455 - lr: 0.0100
Epoch 4/100
31/31 [==============================] - 16s 523ms/step 

In [18]:
def read_image_new(path):
    
    x = cv2.imread(path , cv2.IMREAD_COLOR)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256 , 3
    return x


def read_mask_new(path):

    x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (256, 256))
    # size is 256, 256
    x = np.expand_dims(x, axis = -1)
    # size is 256, 256 , 1
    
    return x

def mask_parse(mask):
    
    mask = np.squeeze(mask)
    mask = [mask , mask , mask]
    
    mask = np.transpose(mask , (1,2,0))
    return mask

In [19]:
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope



In [20]:
if __name__ == "__main__":
    
    test_dataset = tf_dataset(test_x , test_y , batch = batch)
    test_steps = len(test_x) // batch
    
    if len(test_x)%batch != 0 :
        test_steps +=1
        
    with CustomObjectScope({'iou' : iou , 'dice_coef' : dice_coef ,  'dice_coef_loss' :dice_coef_loss }):
    
        model = tf.keras.models.load_model("/kaggle/working/model.h5")
        
    model.evaluate(test_dataset , steps = test_steps)
    
    for i , (x,y) in tqdm(enumerate(zip(test_x , test_y)) , total= len(test_x)):
        
        x = read_image_new(x)
        y = read_mask_new(y)
        
        y_pred = model.predict(np.expand_dims(x , axis = 0))
        
        y_pred= y_pred[0] > 0.5
        
        h,w, _ = x.shape
        
        white_line = np.ones((h,10,3)) * 255.0
        
        all_images = [
            
            x*255.0, white_line,
            
            mask_parse(y) , white_line,
            mask_parse(y_pred)*255.0
            
        ]
        
        image = np.concatenate (all_images, axis =1)
        cv2.imwrite(f"/kaggle/working/{i}.png" , image)


4/4 [==============================] - 2s 320ms/step - loss: 0.1020 - acc: 0.9710 - recall: 0.8239 - precision: 0.8919 - iou: 0.7376 - dice_coef: 0.8487


  0%|          | 0/61 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  2%|▏         | 1/61 [00:01<01:46,  1.78s/it]

1/1 [==============================] - 0s 22ms/step


...

 93%|█████████▎| 57/61 [00:06<00:00, 11.44it/s]

1/1 [==============================] - 0s 20ms/step


 97%|█████████▋| 59/61 [00:06<00:00, 11.76it/s]

1/1 [==============================] - 0s 20ms/step


100%|██████████| 61/61 [00:06<00:00,  8.73it/s]


In [21]:
y_pred.shape

(256, 256, 1)

In [22]:
# !zip -r segmentation_model.zip  "/kaggle/input/segmentation-model"

In [23]:
!zip -r attention_cvc_new.zip   "/kaggle/working/"

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/30.png (deflated 5%)
  adding: kaggle/working/26.png (deflated 6%)
  adding: kaggle/working/9.png (deflated 6%)
  adding: kaggle/working/48.png (deflated 7%)
  adding: kaggle/working/28.png (deflated 5%)
  adding: kaggle/working/4.png (deflated 6%)
  adding: kaggle/working/model.h5 (deflated 8%)
  adding: kaggle/working/16.png (deflated 7%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/41.png (deflated 6%)
  adding: kaggle/working/36.png (deflated 8%)
  adding: kaggle/working/0.png (deflated 6%)
  adding: kaggle/working/40.png (deflated 5%)
  adding: kaggle/working/3.png (deflated 5%)
  adding: kaggle/working/17.png (deflated 5%)
  adding: kaggle/working/50.png (deflated 6%)
  adding: kaggle/working/32.png (deflated 9%)
  adding: kaggle/working/20.png (deflated 7%)
  adding: kaggle/working/14.png (deflated 7%)
  adding: kaggle/working/49.png (deflated 7%)
  adding: kaggle/working/13.png (